In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2

In [3]:
df = pd.read_csv('titanic.csv')

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df=df[['Survived','Pclass','Sex','Age','SibSp','Fare','Parch','Embarked']]

In [6]:
df.sample(10)

,Survived,Pclass,Sex,Age,SibSp,Fare,Parch,Embarked
351,0,1,male,NaN,0,35.0000,0,S
705,0,2,male,39.0,0,26.0000,0,S
158,0,3,male,NaN,0,8.6625,0,S
135,0,2,male,23.0,0,15.0458,0,C
148,0,2,male,36.5,0,26.0000,2,S
861,0,2,male,21.0,1,11.5000,0,S
168,0,1,male,NaN,0,25.9250,0,S
161,1,2,female,40.0,0,15.7500,0,S
240,0,3,female,NaN,1,14.4542,0,C
75,0,3,male,25.0,0,7.6500,0,S


In [7]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Fare          0
Parch         0
Embarked      2
dtype: int64

In [8]:
x_train,x_test,y_train,y_test = train_test_split(df[['Pclass','Sex','Age','SibSp','Fare','Parch','Embarked']],df[['Survived']],test_size=0.2)

In [9]:
x_train.head()

,Pclass,Sex,Age,SibSp,Fare,Parch,Embarked
368,3,female,NaN,0,7.7500,0,Q
275,1,female,63.0,1,77.9583,0,S
661,3,male,40.0,0,7.2250,0,C
643,3,male,NaN,0,56.4958,0,S
39,3,female,14.0,1,11.2417,0,C


In [10]:
#1impute transformer
trf1 = ColumnTransformer(
transformers=[
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embraked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough'
)

In [13]:
#2 = OHE trandsformer
trf2 = ColumnTransformer(transformers=[
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough'
 )

In [20]:
#Scaling transformer

In [14]:
trf3 = ColumnTransformer([
    ('Scaler',MinMaxScaler(),slice(0,10))
])


#MinMax was used because we were going to use feature selection

In [24]:
#Feature selection
#trf4 = SelectKBest(score_func=chi2,k=8)

In [16]:
trf5 = DecisionTreeClassifier()


In [25]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf5',trf5)
])

In [18]:
#Alternate way to make pipeline
pipe_Al = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [26]:
pipe.fit(x_train,y_train)

C:\Users\HP\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embraked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('Scaler', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf5', DecisionTreeClassifier())])

In [27]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embraked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('Scaler', MinMaxScaler(), slice(0, 10, None))]),
 'trf5': DecisionTreeClassifier()}

In [28]:
y_pred = pipe.predict(x_test)

In [29]:
y_pred

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0], dtype=int64)

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

0.6424581005586593

In [32]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))